<a href="https://colab.research.google.com/github/Sushmithaa-Pandian/NLP/blob/main/final_sentiment_tweet_eng.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install tensorflow tensorflow_hub spektral
!pip install tensorflow-text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.1/140.1 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 46.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.8/489.8 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 66.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 109.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 90.0 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.15.0
    Uninstalling wrapt-1.15.0:
      Successfully uninstalled wrapt-1.15.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.13.0
    Uninstalling tensorflow-estimator-2.13.0:
      Successfully uninstalled tensorflow-estimator-2.13

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from spektral.layers import GCNConv, GlobalSumPool, GraphSageConv
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
import tensorflow_text as text
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
df=pd.read_csv('/content/gdrive/MyDrive/Models_Sushmithaa/indicpophash.csv',nrows=500)

In [ ]:
df

,Unnamed: 0,id,conversation_id,created_at,date,time,time_zone,user_id,username,name,...,likes_count,hashtags,link,retweet,geo,language,pop_score,label,label_new,pop_score_log
0,0,1.511390e+18,1.511390e+18,2022-04-05 17:05:33,2022-04-05,17:05:33,UTC,9.659076e+08,ganeshhirennava,Ganesh H,...,0.0,"['ಅನಕ್ಷರಸ್ಥರ_ಪಂಚರ್_ಶಾಪ್', 'ಮೊಂಡ್_ಹಲಾಲ್_ಸರ್ಟಿಫಿ...",http://twitter.com/anyuser/status/151138959227...,False,NaN,kn,2.0,2.0,2.0,0.693147
1,1,1.509950e+18,1.509950e+18,2022-04-01 17:45:50,2022-04-01,17:45:50,UTC,1.395262e+09,LaxmiputraDesai,DESAI LAXMIPUTRA,...,0.0,['BoycottHimalaya'],http://twitter.com/anyuser/status/150995017899...,False,NaN,kn,42.0,3.0,4.0,3.737670
2,2,1.509589e+18,1.509589e+18,2022-03-31 17:52:11,2022-03-31,17:52:11,UTC,1.468286e+18,hops_neil,Wilson Morgan ⚡⚡⚡⚡🌩️🌩️🌩️,...,0.0,['BoycottHimalaya'],http://twitter.com/anyuser/status/150958938914...,False,NaN,kn,1.0,1.0,2.0,0.000000
3,3,1.510501e+18,1.510501e+18,2022-04-03 06:12:39,2022-04-03,06:12:39,UTC,1.328367e+18,repalli_siva,#Hyderabadi,...,0.0,['BoycottHimalaya'],http://twitter.com/anyuser/status/151050050887...,False,NaN,te,4.0,2.0,3.0,1.386294
4,4,1.513160e+18,1.513160e+18,2022-04-10 14:20:31,2022-04-10,14:20:31,UTC,1.489971e+18,PrashanthSkp1,Prashanth Skp,...,0.0,"['brand', 'boss']",http://twitter.com/anyuser/status/151315999909...,False,NaN,kn,6.0,3.0,3.0,1.791759
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,495,1.509882e+18,1.509882e+18,2022-04-01 13:15:05,2022-04-01,13:15:05,UTC,1.992027e+08,prajavani,Prajavani,...,3.0,"['China', 'Covid19', 'Coronavirus']",http://twitter.com/anyuser/status/150988204236...,False,NaN,kn,3.0,2.0,3.0,1.098612
496,496,1.509817e+18,1.509817e+18,2022-04-01 08:56:56,2022-04-01,08:56:56,UTC,1.499715e+18,Darshan07876299,Darshan gowda,...,0.0,['Bengaluru'],http://twitter.com/anyuser/status/150981707558...,False,NaN,kn,1.0,1.0,2.0,0.000000
497,497,1.509816e+18,1.509816e+18,2022-04-01 08:51:04,2022-04-01,08:51:04,UTC,1.499714e+18,Mithun96912107,Mithun gowda,...,0.0,"['Paytm', 'Covid19']",http://twitter.com/anyuser/status/150981559738...,False,NaN,kn,1.0,1.0,2.0,0.000000
498,498,1.509707e+18,1.509707e+18,2022-04-01 01:37:40,2022-04-01,01:37:40,UTC,2.915851e+08,AsianetNewsSN,Asianet Suvarna News,...,1.0,"['SupremeCourt', 'UPSC', 'Coronavirus', 'Covid...",http://twitter.com/anyuser/status/150970653170...,False,NaN,kn,1.0,1.0,2.0,0.000000


In [ ]:
from sklearn.preprocessing import minmax_scale
import pandas as pd

df["class"]= minmax_scale(df["pop_score_log"])

y=df["class"].mean()
df['category']=df['class'].apply(lambda x: 1 if x>=y else 0)
df['hashtags'] = df['hashtags'].str.lower()

In [ ]:
import pandas as pd
from collections import Counter


# Combine all hashtags from all tweets into a single list
all_hashtags = []
for hashtags in df['hashtags']:
    hashtags_list = hashtags.split()  # Assuming hashtags are separated by spaces
    all_hashtags.extend(hashtags_list)

# Count the occurrences of each hashtag across the entire dataset
hashtags_count = Counter(all_hashtags)

# Create a dictionary of hashtag weights based on the occurrences
hashtag_weights = {tag: count for tag, count in hashtags_count.items()}

# Function to calculate the sum of weights for each tweet
def calculate_weight_sum(tweet_hashtags):
    return sum(hashtag_weights.get(tag, 0) for tag in tweet_hashtags.split())

# Add a new column to the DataFrame with the sum of weights
df['hashtag_weight'] = df['hashtags'].apply(calculate_weight_sum)



In [ ]:
df['retweet']=df['retweet'].apply(lambda x: 0 if x=='FALSE' else 1)

In [ ]:
df['no.of_hashtags'] = df['hashtags'].str.split(',').apply(len)


In [ ]:
df['no.of_mentions'] = df['mentions'].str.split(',').apply(len)

In [ ]:
df['no.of_words'] = df['tweet'].str.split(' ').apply(len)

In [ ]:
df['no.of_photos'] = df['photos'].str.split('jpg').apply(len)-1

In [ ]:
user_tweet_counts = df.groupby('user_id').size().reset_index(name='tweet_per_user')
df = df.merge(user_tweet_counts, on='user_id', how='left')

In [ ]:
df.head()

,Unnamed: 0,id,conversation_id,created_at,date,time,time_zone,user_id,username,name,...,weight_sum_dig,class,category,no.of_hashtags,no.of_mentions,no.of_words,no.of_photos,tweet_per_user_x,hashtag_weight,tweet_per_user_y
0,0,1.511390e+18,1.511390e+18,2022-04-05 17:05:33,2022-04-05,17:05:33,UTC,9.659076e+08,ganeshhirennava,Ganesh H,...,5,0.141094,0,5,1,12,0,1,5,1
1,1,1.509950e+18,1.509950e+18,2022-04-01 17:45:50,2022-04-01,17:45:50,UTC,1.395262e+09,LaxmiputraDesai,DESAI LAXMIPUTRA,...,3,0.760825,1,1,1,16,0,1,3,1
2,2,1.509589e+18,1.509589e+18,2022-03-31 17:52:11,2022-03-31,17:52:11,UTC,1.468286e+18,hops_neil,Wilson Morgan ⚡⚡⚡⚡🌩️🌩️🌩️,...,3,0.000000,0,1,1,12,1,1,3,1
3,3,1.510501e+18,1.510501e+18,2022-04-03 06:12:39,2022-04-03,06:12:39,UTC,1.328367e+18,repalli_siva,#Hyderabadi,...,3,0.282188,1,1,1,16,0,1,3,1
4,4,1.513160e+18,1.513160e+18,2022-04-10 14:20:31,2022-04-10,14:20:31,UTC,1.489971e+18,PrashanthSkp1,Prashanth Skp,...,3,0.364723,1,2,2,24,0,1,3,1


In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 55.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 104.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.7 MB/s eta 0:00:00


In [ ]:
dt,dx=train_test_split(df,test_size=0.2,random_state=18)

In [ ]:
dt=dt.reset_index()

In [ ]:
dx=dx.reset_index()

In [ ]:
import pandas as pd
import numpy as np
from transformers import BertTokenizer, BertForSequenceClassification
import torch

tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=3)  # Set num_labels to 3

sentiment_probs = []

for tweet in dt['tweet']:
    inputs = tokenizer(tweet, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    softmax_probs = torch.softmax(logits, dim=-1).cpu().detach().numpy()[0]
    sentiment_probs.append(softmax_probs)


S_train = np.array(sentiment_probs)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
sentiment_probs = []
for tweet in dx['tweet']:

    inputs = tokenizer(tweet, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    softmax_probs = torch.softmax(logits, dim=-1).cpu().detach().numpy()[0]
    sentiment_probs.append(softmax_probs)

S_test = np.array(sentiment_probs)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

def adjacency_matrix(dataframe):
    num_rows, num_cols = len(dataframe), len(dataframe)
    adjacency_matrix = np.zeros((num_rows, num_cols))

    for i in range(num_rows):
        for j in range(i, num_cols):
            common_tags = len(set(dataframe['hashtags'][i]) & set(dataframe['hashtags'][j]))
            adjacency_matrix[i, j] = common_tags
            adjacency_matrix[j, i] = common_tags
    scaler = MinMaxScaler()

    adjacency_matrix = scaler.fit_transform(adjacency_matrix)


    adj_sparse = tf.convert_to_tensor(adjacency_matrix, dtype=tf.float32)
    adjacency_matrix_sparse = tf.sparse.from_dense(adj_sparse)

    return adjacency_matrix_sparse

In [ ]:
adjacency_matrix_sparse_train=adjacency_matrix(dt)


In [ ]:
adjacency_matrix_sparse_train.shape

TensorShape([400, 400])

In [ ]:
adjacency_matrix_sparse_test=adjacency_matrix(dx)

In [ ]:
adjacency_matrix_sparse_test.shape

TensorShape([100, 100])

In [ ]:
bert_preprocessor_url = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
bert_encoder_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4"

bert_preprocessor = hub.KerasLayer(bert_preprocessor_url)
bert_encoder = hub.KerasLayer(bert_encoder_url)
def create_bert_encodings(texts):
    input_tokens = bert_preprocessor(texts)
    encoder_inputs = dict(input_tokens)
    encoder_outputs = bert_encoder(encoder_inputs)["pooled_output"]
    return encoder_outputs

In [ ]:
X_train=dt["tweet"].values

In [ ]:
Y_train=np.asarray(dt['category'].values).astype('float32')

In [ ]:
X_test=dx["tweet"].values

In [ ]:
Y_test=np.asarray(dx['category'].values).astype('float32')

In [ ]:
from tensorflow.keras.layers import Input, Concatenate, Dense

In [ ]:
train_enc=create_bert_encodings(X_train)

In [ ]:
additional_features=dt[["retweet_count",'likes_count','replies_count','no.of_hashtags', 'no.of_mentions', 'no.of_words',
       'no.of_photos','tweet_per_user_x','hashtag_weight']].values

In [ ]:
concatenated_features = Concatenate(axis=-1)([additional_features,S_train,train_enc])

In [ ]:
concatenated_features.shape

TensorShape([400, 780])

In [ ]:
gsc_enc=GraphSageConv(channels=780)([concatenated_features,adjacency_matrix_sparse_train])

In [ ]:
from tensorflow import keras
model = keras.Sequential()
model.add(keras.layers.Dense(384, activation='relu', input_shape=(780,)))
model.add(keras.layers.Dropout(0.2))  # Dropout layer with a dropout rate of 0.2
model.add(keras.layers.Dense(192, activation='relu'))
model.add(keras.layers.Dropout(0.2))  # Dropout layer with a dropout rate of 0.2
model.add(keras.layers.Dense(96, activation='relu'))
model.add(keras.layers.Dropout(0.2))  # Dropout layer with a dropout rate of 0.2
model.add(keras.layers.Dense(48, activation='relu'))
model.add(keras.layers.Dropout(0.2))  # Dropout layer with a dropout rate of 0.2
model.add(keras.layers.Dense(24, activation='relu'))
model.add(keras.layers.Dropout(0.2))  # Dropout layer with a dropout rate of 0.2
model.add(keras.layers.Dense(12, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 384)               299904    
                                                                 
 dropout (Dropout)           (None, 384)               0         
                                                                 
 dense_1 (Dense)             (None, 192)               73920     
                                                                 
 dropout_1 (Dropout)         (None, 192)               0         
                                                                 
 dense_2 (Dense)             (None, 96)                18528     
                                                                 
 dropout_2 (Dropout)         (None, 96)                0         
                                                                 
 dense_3 (Dense)             (None, 48)                4

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(gsc_enc,Y_train, epochs=10, batch_size=Y_train.shape[0])


Epoch 1/10
1/1 [==============================] - 13s 13s/step - loss: 0.6937 - accuracy: 0.4425
Epoch 2/10
1/1 [==============================] - 0s 14ms/step - loss: 0.6937 - accuracy: 0.4625
Epoch 3/10
1/1 [==============================] - 0s 15ms/step - loss: 0.6934 - accuracy: 0.4700
Epoch 4/10
1/1 [==============================] - 0s 17ms/step - loss: 0.6936 - accuracy: 0.4725
Epoch 5/10
1/1 [==============================] - 0s 16ms/step - loss: 0.6930 - accuracy: 0.5200
Epoch 6/10
1/1 [==============================] - 0s 19ms/step - loss: 0.6924 - accuracy: 0.5700
Epoch 7/10
1/1 [==============================] - 0s 28ms/step - loss: 0.6921 - accuracy: 0.5700
Epoch 8/10
1/1 [==============================] - 0s 31ms/step - loss: 0.6919 - accuracy: 0.5600
Epoch 9/10
1/1 [==============================] - 0s 17ms/step - loss: 0.6915 - accuracy: 0.5650
Epoch 10/10
1/1 [==============================] - 0s 19ms/step - loss: 0.6914 - accuracy: 0.5750


In [ ]:
test_enc=create_bert_encodings(X_test)

In [ ]:
additional_features_t=dx[["retweet_count",'likes_count','replies_count','no.of_hashtags', 'no.of_mentions', 'no.of_words',
       'no.of_photos',"tweet_per_user_x",'hashtag_weight']].values

In [ ]:
concatenated_features_t = Concatenate(axis=-1)([additional_features_t,S_test,test_enc])

In [ ]:
gsc_enc_t=GraphSageConv(channels=780)([concatenated_features_t,adjacency_matrix_sparse_test])

In [ ]:
y_pred=model.predict(gsc_enc_t)

4/4 [==============================] - 0s 5ms/step


In [ ]:
y_pred

array([[0.49253783],
       [0.49260956],
       [0.49260882],
       [0.4927348 ],
       [0.49210185],
       [0.49269766],
       [0.492591  ],
       [0.49233815],
       [0.49271512],
       [0.49249777],
       [0.49213302],
       [0.49295178],
       [0.49241677],
       [0.49284267],
       [0.49272144],
       [0.4924641 ],
       [0.49262142],
       [0.49262902],
       [0.49266437],
       [0.49227685],
       [0.49253425],
       [0.4927135 ],
       [0.49219584],
       [0.49248394],
       [0.49242255],
       [0.4925298 ],
       [0.49283272],
       [0.49258804],
       [0.4926581 ],
       [0.49268496],
       [0.4928975 ],
       [0.49253154],
       [0.49269018],
       [0.49276787],
       [0.49251777],
       [0.49251518],
       [0.4925415 ],
       [0.49256712],
       [0.49265176],
       [0.49280214],
       [0.49273828],
       [0.49270603],
       [0.4925466 ],
       [0.49247676],
       [0.4925552 ],
       [0.49243024],
       [0.49263358],
       [0.492

In [ ]:
y_pred=np.where(y_pred>y,1,0)

In [ ]:
precision_score(Y_test,y_pred)

0.51

In [ ]:
recall_score(Y_test,y_pred)

1.0

In [ ]:
f1_score(Y_test,y_pred)

0.6754966887417219

In [ ]:
accuracy_score(Y_test,y_pred)

0.51

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(Y_test,y_pred))

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00        49
         1.0       0.51      1.00      0.68        51

    accuracy                           0.51       100
   macro avg       0.26      0.50      0.34       100
weighted avg       0.26      0.51      0.34       100



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
